### Define and Train a RNN 
to Predict a binary variable (1 or 0), whether a voltage rise (above a predefined threshold i.e. 1.0250 ) will occur or not in the next period

Prediction using the created RNN is done here [voltageRiseBin_Pred](voltageRiseBin_Pred.ipynb)

---

#### Import modules to be used

In [2]:
#Import Modules 
import pandas as pd
import pandapower as pp
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm # Profiling 
import seaborn as sbn
import pickle, sys, importlib,  time
import os
from pickle import load
import tensorflow as tf
import joblib

#### Import Module for ML

In [3]:
# import pakages forML
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping

#### Import my own modules

In [10]:
# import psutil
function_folder = '../Modules/' 
# Add function folder to path if it is not already
if function_folder not in sys.path: sys.path.append(function_folder)

import oriFunctions as oriFc
from oriVariables import (network_folder,
                          simResult_folder,
                          defAuth_hvBus_vRiseMax, 
                          defAuth_hvBus_vRiseMin, 
                          excel_folder, 
                          trainSet_end_M1, 
                          valSet_end_M2,
                          hm10_start_end, 
                          h_start_end, 
                          pd_Δt, 
                          cleanedData_name)

#### Import data Cleaned file for training the RNN

In [5]:
df_data = joblib.load(simResult_folder+cleanedData_name)

# Extract only a part of data to be used as 
df_data = df_data[df_data.index<=valSet_end_M2]

In [10]:
# # Extract only the relavant testing set since the training set covers the first part of the data
df_final = df_data[df_data.index < trainSet_end_M1]

# Extract only daylight period i.e. from 07am to 7PM
# The daylight period is considered to be defined between 07am and 7Pm excluded. 
per_index = df_final.index
per_daylight = (pd.Series(index=per_index.to_timestamp(), dtype=object)
                .between_time(*hm10_start_end) ).index.to_period(pd_Δt)

# Extract only daylight hours 
df_final = df_final.loc[per_daylight]


per_index = df_final.index
per_index2 = ( pd.Series(index=per_index.to_timestamp(), dtype=object
                           ).between_time(*h_start_end) ).index.to_period(pd_Δt)

# defined Vrise
vm_mu_max, vm_mu_min = defAuth_hvBus_vRiseMax, defAuth_hvBus_vRiseMin  

#### Import the voltage rise from [VoltageRiseBinaryUpdated](VoltageRiseBinaryUpdated.ipynb)


In [18]:
# Import the voltage rise from 
numAndBin_vRise = joblib.load(simResult_folder+'Binary_Voltage_Rise.pkl')
df_final['Volt_Rise_bin'] = numAndBin_vRise['Volt_Rise_Bin']

### Set variables For binary voltage rise prediction

In [19]:
# I'm using all the dataset to train the RNN to improve the performance since ive already
# tried with the testing set and get an accuraccy of 94%
# # Separate training and testing set 
df_train = df_final[df_final.index<trainSet_end_M1]

# Define scaler
numerical_scaler = MinMaxScaler()
numerical_scaler.fit(df_train);

# train_scaled, test_scaled= map(numerical_scaler.transform, [df_train, df_test])
train_scaled = numerical_scaler.transform(df_train)

##### Define Timeseries  generators


In [20]:
gen_length = 6 # 1 hour

batchSize = (gen_length//6)*24*7; #  (gen_length//6)  To convert in hour *24 hour * 7 days

train_generator = TimeseriesGenerator(train_scaled, train_scaled[:,-1], 
                                      length = gen_length, 
                                      batch_size= batchSize)

# test_genarator = TimeseriesGenerator(test_scaled, test_scaled[:,-1], 
#                                      length = gen_length, 
#                                      batch_size= batchSize)

n_features = train_generator[0][0][0].shape[1]  # Define total number of features

#### Define RNN

In [21]:
bin_vRise_RNN = Sequential()

bin_vRise_RNN.add( LSTM(units=128, activation='tanh', input_shape=(gen_length,n_features), return_sequences=True) )
bin_vRise_RNN.add( LSTM(units=128, activation='relu') )
bin_vRise_RNN.add(Dense(units=1, activation='sigmoid'))

bin_vRise_RNN.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

##### Define early stopping mechanism

In [22]:
# early_stop = EarlyStopping(monitor= 'val_loss',patience=30, mode='min')

#### Train RNN

In [23]:
bin_vRise_RNN.fit(train_generator, 
          epochs=80)

Epoch 1/80
225/225 [==============================] - 5s 15ms/step - loss: 0.1896 - accuracy: 0.9355
Epoch 2/80
225/225 [==============================] - 3s 14ms/step - loss: 0.0958 - accuracy: 0.9645
Epoch 3/80
225/225 [==============================] - 3s 15ms/step - loss: 0.0832 - accuracy: 0.9705
Epoch 4/80
225/225 [==============================] - 3s 15ms/step - loss: 0.0813 - accuracy: 0.9711
Epoch 5/80
225/225 [==============================] - 3s 14ms/step - loss: 0.0793 - accuracy: 0.9710
Epoch 6/80
225/225 [==============================] - 4s 17ms/step - loss: 0.0793 - accuracy: 0.9706
Epoch 7/80
225/225 [==============================] - 4s 20ms/step - loss: 0.0785 - accuracy: 0.9717
Epoch 8/80
225/225 [==============================] - 5s 20ms/step - loss: 0.0771 - accuracy: 0.9711
Epoch 9/80
225/225 [==============================] - 4s 19ms/step - loss: 0.0759 - accuracy: 0.9716
Epoch 10/80
225/225 [==============================] - 4s 18ms/step - loss: 0.0780 - accura

In [24]:
bin_vRise_RNN.save(f'{network_folder}RNN/StLaurent_bin_vRise_model')
joblib.dump(numerical_scaler, f'{network_folder}RNN/StLaurent_bin_vRise_scaler.plk')

INFO:tensorflow:Assets written to: ../Pickle_files/RNN/StLaurent_bin_vRise_model\assets


INFO:tensorflow:Assets written to: ../Pickle_files/RNN/StLaurent_bin_vRise_model\assets


['../Pickle_files/RNN/StLaurent_bin_vRise_scaler.plk']